# General rectangular guides in McStasScript
This notebook shows how to create neutron guides from arbitrary python functions from many rectangular segments. It uses McStasScript to create the instrument and a function to add the optics.

In [ ]:
import mcstasscript as ms
import numpy as np

This functions adds the optics given:
- instrument object: McStasScript instrument object
- name: str
- point_array: numpy array describing segment start and end points
- m_array: float or numpy array with m coatings for each segment, length one less than point_array
- reference: reference to place guide relative too, default ABSOLUTE
- width: python function describing the width of the guide as a function of point_array distance
- height: python function describing the height
- top: python function describing top (overwritten by height)
- left: python function describing left side (overwritten by width)
- right: python function describing right side (overwritten by width)
- bottom: python function describing bottom (overwritten by height)

This input scheme allows the user to choose between giving width or left/right side depending on whether the guide is symetric or not.

In [ ]:
def add_optics(instrument, name, point_array, m_array, reference="ABSOLUTE", width=None, height=None,
               top=None, left=None, right=None, bottom=None):
    
    if width is not None:
        # If this is given, overwrite left / right
        left = lambda z : -0.5*width(z)
        right = lambda z : 0.5*width(z)

    if height is not None:
        # If this is given, overwrite top / bottom
        top = lambda z : 0.5*height(z)
        bottom = lambda z : -0.5*height(z)
        
    # Ensure sufficient information is given by the user
    if top is None  or left is None or right is None or bottom is None:
        raise ValueError("Not sufficient input to construct optics.")
    
    # If m_array is just a number, create the appropriate array
    if isinstance(m_array, (float, int)):
        m_array = m_array*np.ones(len(point_array) - 1)
        
    # Produce arrays with start and end points, each shorter than the point_array, now corresponding to segments
    point_array_starts = point_array[0:-1]
    point_array_ends = point_array[1:]
    
    # Width corresponds to x in mcstas, but rotations around y
    start_mid_xs = 0.5*(left(point_array_starts) + right(point_array_starts))
    end_mid_xs = 0.5*(left(point_array_ends) + right(point_array_ends))
    rotation_ys = np.arctan((start_mid_xs - end_mid_xs)/(point_array_ends-point_array_starts))*180/np.pi
    rotation_ys = np.append(rotation_ys, 0)
    start_widths = right(point_array_starts) - left(point_array_starts)
    end_widths = right(point_array_ends) - left(point_array_ends)

    # Heights corresponds to y in mcstas, but rotations around x
    start_mid_ys = 0.5*(top(point_array_starts) + bottom(point_array_starts))        
    end_mid_ys = 0.5*(top(point_array_ends) + bottom(point_array_ends))
    rotation_xs = np.arctan((start_mid_ys - end_mid_ys)/(point_array_ends-point_array_starts))*180/np.pi
    rotation_xs = np.append(rotation_xs, 0)
    start_heights = top(point_array_starts) - bottom(point_array_starts)
    end_heights = top(point_array_ends) - bottom(point_array_ends)

    # Add segments
    for index in range(len(point_array_starts)):    
        start = point_array_starts[index]
        end = point_array_ends[index]
        
        start_mid_x = start_mid_xs[index]
        end_mid_x = end_mid_xs[index]
        rotation_y = rotation_ys[index]
        next_rotation_y = rotation_ys[index+1]        
        effective_rotation_y = next_rotation_y - rotation_y        
        start_width = start_widths[index]
        end_width = end_widths[index]

        start_mid_y = start_mid_ys[index]
        end_mid_y = end_mid_ys[index]
        rotation_x = rotation_xs[index]
        next_rotation_x = rotation_xs[index+1]
        effective_rotation_x = next_rotation_x - rotation_x
        start_height = start_heights[index]
        end_height = end_heights[index]
        
        length = end - start        
        assert length > 0

        # Calculate a safe distance to avoid having mirrors overlapping in corners due to rotation
        safe_distance_x = start_height*np.sin(effective_rotation_x*np.pi/180)
        safe_distance_y = start_width*np.sin(effective_rotation_y*np.pi/180)
        safe_distance = np.max([safe_distance_x, safe_distance_y])
        
        assert safe_distance < length
                
        # Add Guide_gravity component to instrument file
        comp = instrument.add_component(f"general_optics_{name}_{index}", "Guide_gravity")
        comp.set_parameters(w1=start_width, w2=end_width, h1=start_height, h2=end_height,
                            l=length-safe_distance-1E-6, m=m_array[index])
        comp.set_AT([start_mid_x, start_mid_y, start], RELATIVE=reference)
        comp.set_ROTATED([rotation_x, -rotation_y, 0], RELATIVE=reference)

## Create the McStas instrument
A McStasScript instrument is made to demonstrate the new functionality. First we create the instrument object.

In [ ]:
instrument = ms.McStas_instr("optics")

Next we add a simple source

In [ ]:
source = instrument.add_component("source", "Source_simple")
source.set_parameters(xwidth=0.1, yheight=0.1, lambda0=5.0, dlambda=2.0)

Define the input for the *add_optics* function:
- Array with start points, here from 2 to 8 m from the source
- Side function with sinus wave to create an interesting shape
- Translated version to be the other side

It is important that the functions can take and return a numpy array.

In [ ]:
points = np.linspace(2, 10, 8*5+1)

def side_function(z):
    repetition_length = 8.0
    offset = 2.0
    return 0.01 + 0.05*np.sin((z-offset)*2*np.pi/repetition_length)

def negative_side_function(z):
    return side_function(z) - 0.02 # width of guide

add_optics(instrument=instrument, name="wiggle", point_array=points, m_array=4.0,
           left=negative_side_function, right=side_function, bottom=negative_side_function, top=side_function)

focus_x = abs(negative_side_function(points[0]) - side_function(points[0]))
focus_y = abs(negative_side_function(points[0]) - side_function(points[0]))
# Set focusing parameters of source to aim at the first element
source.set_parameters(focus_xw=focus_x, focus_yh=focus_y, target_index=1)

Show instrument to ensure that the geomery is as desired.

In [ ]:
instrument.show_instrument()

Add a simple monitor after the element to see the transported beam.

In [ ]:
psd = instrument.add_component("PSD", "PSD_monitor")
psd.set_parameters(xwidth=0.1, yheight=0.1, nx=200, ny=200, filename='"PSD.dat"')
psd.set_AT([0, 0, points[-1]+0.5])

Run a simulation with the *backengine* method.

In [ ]:
instrument.settings(5*1E6)
data = instrument.backengine()

Plot the resulting data to see the beam could be transferred through the guide.

In [ ]:
ms.make_sub_plot(data)